In [ ]:
## https://huggingface.co/transformers/model_doc/bert.html
## https://huggingface.co/transformers/pretrained_models.html

### data_loader 的MyPro加载数据
### bert_ner 中 有模型

### 检测模型使用

In [ ]:
from data_loader import MyPro,convert_examples_to_features,create_batch_iter
from bert_ner import Bert_Softmax
# from bert_ner import Bert_Softmax
import torch
from transformers import BertConfig,AdamW

import torch.nn as nn

import torch.optim as optim

In [ ]:
from Evalutor import evaluate

In [ ]:
label_list = ['O', 'B-Nh', 'I-Nh', 'B-NDR', 'I-NDR']
train_iter = create_batch_iter("train",'../multi_data/multi_trainner.txt',12,label_list)

config = BertConfig.from_json_file('../bert-base-chinese/bert_config.json')
bert_softmax = Bert_Softmax(config,5).cuda()

In [ ]:
weight = [10.0] * len(label_list)   #生成一维

weight[0] = 1

weight = torch.tensor(weight).cuda()

criterion = nn.NLLLoss(weight,size_average=False)      #定义的损失函数，softmax以后求损失函数

In [ ]:
# param_optimizer = bert_softmax.named_parameters()

# param_optimizer = list(bert_softmax.named_parameters())
# no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']

# optimizer_grouped_parameters = [
#         {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
#         {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]

# optimizer = AdamW(optimizer_grouped_parameters,lr = 0.00001)

In [ ]:
### 冻结部分参数

unfreeze_layers = ['layer.11','bert.pooler','classifier.']
 
for name, param in bert_softmax.named_parameters():
    print(name,param.size())

print("*"*30)
print('\n')

for name ,param in bert_softmax.named_parameters():
    param.requires_grad = False
    for ele in unfreeze_layers:
        if ele in name:
            param.requires_grad = True
            break
#验证一下
for name, param in bert_softmax.named_parameters():
    if param.requires_grad:
        print(name,param.size())
 
    


In [ ]:
#过滤掉requires_grad = False的参数
#optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, bert_softmax.parameters()), lr=0.1)
lr = 0.0001
optimizer = AdamW(filter(lambda p: p.requires_grad, bert_softmax.parameters()), lr=lr,weight_decay = 0.1)

In [ ]:
# optimizer = optim.SGD(bert_softmax.parameters(), lr = 0.0001, momentum = 0.9)

In [ ]:
batch_size = 8

In [ ]:
bert_softmax.train()
for e in range(5):
    count = 0
    if (e+1)%2 == 0:
        lr = lr/2
        optimizer = AdamW(filter(lambda p: p.requires_grad, bert_softmax.parameters()), lr=lr,weight_decay = 0.1)
    
    for step, batch in enumerate(train_iter):
        count += 1
        input_ids, input_mask, segment_ids, label_ids = batch   ##(batch_size,512)
        input_ids = input_ids.cuda()
        input_mask = input_mask.cuda()
        segment_ids = segment_ids.cuda()
        label_ids = label_ids.cuda()
        out = bert_softmax(input_ids,segment_ids,input_mask)  #[batch_size,seq_len+2(512)]
        
        out1 = out.view(-1, out.size(2))
        res1 = label_ids[0]
        for i in range(1,len(label_ids)):
            res1 = torch.cat((res1,label_ids[i]))
        loss = criterion(out1,res1)
        loss = loss/len(out)
        
        if count %5 == 0:
            print('loss: ',loss)
            out2 = torch.argmax(out,dim=2)
            batch_precision,batch_recall,batch_f1 = evaluate(out2,label_ids,input_mask)
            print('batch_precision:%.4f  batch_recall:%.4f  batch_f1: %.4f' %(batch_precision,batch_recall,batch_f1)) 
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(str(e)+'：结束')
        
        

In [ ]:
## batch_precision:0.9526  batch_recall:0.9731  batch_f1: 0.9628


In [ ]:
## 保存模型
# len(bert_softmax.state_dict())
PATH = '../model_save/bert_softmax'
torch.save(bert_softmax.state_dict(), PATH)

In [ ]:
### load model
config = BertConfig.from_json_file('../bert-base-chinese/bert_config.json')
# bert_softmax = Bert_Softmax(config,5).cuda()

In [ ]:
label_list = ['O', 'B-Nh', 'I-Nh', 'B-NDR', 'I-NDR']
eval_iter = create_batch_iter("train",'../multi_data/multi_testner.txt',8,label_list)

In [ ]:
### load_model
PATH = '../model_save/bert_softmax'
model = Bert_Softmax(config,5).cuda()
model.load_state_dict(torch.load(PATH))
model.eval()

#### load_model 
# out = model(input_ids,segment_ids,input_mask)  
        
# out1 = out.view(-1, out.size(2))
# res1 = label_ids[0]
# for i in range(1,len(label_ids)):
#     res1 = torch.cat((res1,label_ids[i]))
# loss = criterion(out1,res1)
# loss = loss/len(out)

# if count %5 == 0:
# print('loss: ',loss)
# out2 = torch.argmax(out,dim=2)
# batch_precision,batch_recall,batch_f1 = evaluate(out2,label_ids,input_mask)
# print('batch_precision:%.4f  batch_recall:%.4f  batch_f1: %.4f' %(batch_precision,batch_recall,batch_f1)) 

In [ ]:
weight = [10.0] * len(label_list)   #生成一维

weight[0] = 1

weight = torch.tensor(weight).cuda()

criterion = nn.NLLLoss(weight,size_average=False)      #定义的损失函数，softmax以后求损失函数

In [ ]:
model.eval()

In [ ]:
eval_iter = create_batch_iter("train",'../multi_data/multi_testner.txt',8,label_list)
# bert_softmax.train()
for step, batch in enumerate(eval_iter):
#     count += 1
    input_ids, input_mask, segment_ids, label_ids = batch   ##(batch_size,512)
    input_ids = input_ids.cuda()
    input_mask = input_mask.cuda()
    segment_ids = segment_ids.cuda()
    label_ids = label_ids.cuda()
    out = model(input_ids,segment_ids,input_mask)  #[batch_size,seq_len+2(512)]

    out1 = out.view(-1, out.size(2))
    res1 = label_ids[0]
    for i in range(1,len(label_ids)):
        res1 = torch.cat((res1,label_ids[i]))
    loss = criterion(out1,res1)
    loss = loss/len(out)

    
    print('loss: ',loss)
    out2 = torch.argmax(out,dim=2)
    batch_precision,batch_recall,batch_f1 = evaluate(out2,label_ids,input_mask)
    print('batch_precision:%.4f  batch_recall:%.4f  batch_f1: %.4f' %(batch_precision,batch_recall,batch_f1)) 


In [ ]:
from collections import Counter 

In [ ]:
Counter(input_mask[3].tolist())

In [ ]:
Counter(label_ids[3].tolist())

In [ ]:
Counter(input_ids[3].tolist())

In [ ]:
    #     label_true = []
    #     y_pre = []
#         loss = torch.tensor(0.0,requires_grad=True)
#         for i in range(len(out)):
#     #         label_true.append([label_ids[i][j].item() for j in range(512) if input_mask[i][j] == 1 ])
#     #         y_pre.append([out[i][j].tolist() for j in range(512) if input_mask[i][j] == 1 ])
            
#             label_true = [label_ids[i][j].item() for j in range(512) if input_mask[i][j] == 1 ]
#             y_pre = [out[i][j].tolist() for j in range(512) if input_mask[i][j] == 1 ]

#             assert len(label_true) == len(y_pre)

#             label_true = torch.tensor(label_true).cuda()
#             y_pre = torch.tensor(y_pre).cuda()

#             tmp_loss = criterion(y_pre, label_true)
# #             tmp_loss = tmp_loss/len(y_pre)
#             loss = loss+tmp_loss
# #             loss.backward()
# #             optimizer.backward(loss)
    
    
#         break

# BERT-RE

In [1]:
# from redata_loader import MyPro,convert_examples_to_features,create_batch_iter
from redata_loader import create_batch_iter
from bert_ner import RE_Bert_Softmax
# from bert_ner import Bert_Softmax
import torch
from transformers import BertConfig,AdamW

import torch.nn as nn

import torch.optim as optim

In [2]:
import torch.nn.functional as F

In [3]:
tag2id = {}
tag2id['O'] = 0
tag2id['traffic_in'] = 1
tag2id['sell_drugs_to'] = 2
tag2id['provide_shelter_for'] = 3
tag2id['posess'] = 4

In [4]:
train_iter = create_batch_iter("train",'../multi_data/multi_trainre.txt',12,tag2id)

In [5]:
config = BertConfig.from_json_file('../bert-base-chinese/bert_config.json')
bert_softmax = RE_Bert_Softmax(config,5).cuda()

In [6]:
# weight = [10.0] * len(tag2id)   #生成一维
# weight[0] = 1
# weight = torch.tensor(weight).cuda()

criterion = nn.NLLLoss(size_average=False)      #定义的损失函数，softmax以后求损失函数

/home/irsdc/songwenhui/anaconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [7]:
### 冻结部分参数

unfreeze_layers = ['layer.11','bert.pooler','classifier.']
 
for name, param in bert_softmax.named_parameters():
    print(name,param.size())

print("*"*30)
print('\n')

for name ,param in bert_softmax.named_parameters():
    param.requires_grad = False
    for ele in unfreeze_layers:
        if ele in name:
            param.requires_grad = True
            break
#验证一下
for name, param in bert_softmax.named_parameters():
    if param.requires_grad:
        print(name,param.size())

bert.embeddings.word_embeddings.weight torch.Size([21128, 768])
bert.embeddings.position_embeddings.weight torch.Size([512, 768])
bert.embeddings.token_type_embeddings.weight torch.Size([2, 768])
bert.embeddings.LayerNorm.weight torch.Size([768])
bert.embeddings.LayerNorm.bias torch.Size([768])
bert.encoder.layer.0.attention.self.query.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.query.bias torch.Size([768])
bert.encoder.layer.0.attention.self.key.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.key.bias torch.Size([768])
bert.encoder.layer.0.attention.self.value.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.value.bias torch.Size([768])
bert.encoder.layer.0.attention.output.dense.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.output.dense.bias torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.weight torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.bias torch.Size([768])
bert.encoder

In [8]:
#过滤掉requires_grad = False的参数
#optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, bert_softmax.parameters()), lr=0.1)
lr = 0.0001
optimizer = AdamW(filter(lambda p: p.requires_grad, bert_softmax.parameters()), lr=lr,weight_decay = 0.1)

In [12]:
bert_softmax.train()
for e in range(5):
#     count = 0
    if (e+1)%2 == 0:
        lr = lr/2
        optimizer = AdamW(filter(lambda p: p.requires_grad, bert_softmax.parameters()), lr=lr,weight_decay = 0.1)
    
    for step, batch in enumerate(train_iter):
#         count += 1
        input_ids, input_mask, segment_ids, label_ids = batch   ##(batch_size,512)
        input_ids = input_ids.cuda()
        input_mask = input_mask.cuda()
        segment_ids = segment_ids.cuda()
        label_ids = label_ids.cuda()
        out = bert_softmax(input_ids,segment_ids,input_mask)  #[batch_size,seq_len+2(512)]
        
        loss = F.cross_entropy(out, label_ids)
        
#         out1 = out.view(-1, out.size(2))
#         res1 = label_ids[0]
#         for i in range(1,len(label_ids)):
#             res1 = torch.cat((res1,label_ids[i]))
#         loss = criterion(out,label_ids)
        #loss = loss/len(out)
        
#         if count %5 == 0:
        print('loss: ',loss)
#             out2 = torch.argmax(out,dim=2)
#             batch_precision,batch_recall,batch_f1 = evaluate(out2,label_ids,input_mask)
#             print('batch_precision:%.4f  batch_recall:%.4f  batch_f1: %.4f' %(batch_precision,batch_recall,batch_f1)) 
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(str(e)+'：结束')

loss:  tensor(0.4584, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.5900, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.3645, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.5375, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.3725, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.3418, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.4926, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.6382, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.3073, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.5907, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.7454, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.5296, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.3110, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.4475, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.5272, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  ten

loss:  tensor(0.7092, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.7026, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.7441, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.5097, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.6547, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.4566, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.3629, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.3392, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.7955, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.5292, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.8730, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.6568, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.4103, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.4780, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.4349, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  ten

loss:  tensor(0.6789, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.5653, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.4516, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.3249, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.7753, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.4805, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.5467, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.5649, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.4349, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.5091, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.3506, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.3834, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.8146, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.7869, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.4991, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  ten

loss:  tensor(0.7276, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.3043, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.3046, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.7187, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.6332, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.5175, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.3593, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.8017, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.3915, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.2994, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.4575, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(1.0266, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.5690, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.4155, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.2455, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  ten

loss:  tensor(0.5074, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.6563, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.6872, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.2513, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.6022, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.2300, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.4837, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.5190, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.2820, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.4800, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.7352, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.6079, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.4903, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.4199, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.5156, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  ten

...

loss:  tensor(0.5872, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.2616, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.2571, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.3300, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.5714, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.4990, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.2473, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.5375, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.3925, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.3154, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.2793, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.3002, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.5484, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.3424, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.2983, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  ten

loss:  tensor(0.2840, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.5434, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.4513, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.1943, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.1433, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.4845, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.2478, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.3162, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.2615, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.3527, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.3434, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.5068, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.5005, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.2435, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.2141, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  ten

loss:  tensor(0.4289, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.2783, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.3677, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.2411, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.5424, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.1916, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.3223, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.2908, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.2882, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.2614, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.2789, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.2920, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.4403, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.3123, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.6684, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  ten

loss:  tensor(0.2731, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.4708, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.4051, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.3452, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.1617, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.1691, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.3635, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.9378, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.5189, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.4684, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.2048, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.2783, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.1678, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.4411, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.3544, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  ten

loss:  tensor(0.0803, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.0965, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.3215, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.3926, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.2125, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.4995, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.4846, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.3969, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.5381, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.3134, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.4377, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.2366, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.3202, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.1841, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  tensor(0.2720, device='cuda:0', grad_fn=<NllLossBackward>)
loss:  ten

In [ ]:
out.shape

In [ ]:
label_ids.shape